In [1]:
import numpy as np
import pandas as pd

In [2]:
# pd.set_option('display.max_columns', None)
df = pd.read_csv('../data/cleaned_review_data.csv', 
                 parse_dates=['SubmissionTime', 'FirstSubmissionTime', 'LastSubmissionTime'], 
                 low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1364686 entries, 0 to 1364685
Data columns (total 24 columns):
 #   Column                Non-Null Count    Dtype         
---  ------                --------------    -----         
 0   pd_id                 1364686 non-null  object        
 1   brand                 1364686 non-null  object        
 2   Name                  1364686 non-null  object        
 3   Description           1364686 non-null  object        
 4   AverageOverallRating  1364686 non-null  float64       
 5   love_count            1364686 non-null  float64       
 6   reviews_count         1364686 non-null  float64       
 7   Price                 1364686 non-null  float64       
 8   category_1            1364686 non-null  object        
 9   category_2            1006831 non-null  object        
 10  category_3            957356 non-null   object        
 11  FirstSubmissionTime   1364686 non-null  datetime64[ns]
 12  LastSubmissionTime    1364686 non-null  da

In [58]:
pds = pd.read_csv('../data/cleaned_pd_data.csv', parse_dates=['FirstSubmissionTime', 'LastSubmissionTime'])
pds.drop(columns=['Age_35to44', 'Age_18to24', 'Age_25to34', 'Age_45to54', 
                  'Age_over54', 'Age_13to17', 'category', 'size_oz', 'size_mL', 'size_g'], 
         inplace=True)
pds.loc[pds['category_3'].isna(), 'category_3'] = pds['category_2']
pds.loc[pds['category_3'].isna(), 'category_3'] = pds['category_1']
pds.rename(columns={'category_3': 'category'}, inplace=True)

In [59]:
pds_c = pds.copy()
pds_c['category'] = pds_c['category'].str.lstrip()
pds_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2301 entries, 0 to 2300
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   brand                 2301 non-null   object        
 1   pd_id                 2301 non-null   object        
 2   Name                  2301 non-null   object        
 3   Description           2301 non-null   object        
 4   AverageOverallRating  2301 non-null   float64       
 5   FirstSubmissionTime   2301 non-null   datetime64[ns]
 6   LastSubmissionTime    2301 non-null   datetime64[ns]
 7   love_count            2301 non-null   float64       
 8   reviews_count         2301 non-null   float64       
 9   Price                 2301 non-null   float64       
 10  category_1            2301 non-null   object        
 11  category_2            1765 non-null   object        
 12  category              2301 non-null   object        
dtypes: datetime64[ns](

In [ ]:
def recommender(user_id, pd_id):
    """
    """
    
#     return top_5_sim_pd, top_5_you_may_like

## Product similarity

Brand / Categories / Price / Description

In [81]:
import sklearn
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel, euclidean_distances

In [82]:
def compute_similarity(col):
    """
    Function to compute cosine similarity based on TFIDF.
    Input: 
        col - The text column for TFIDF analysis
    """
    stopwords_list = stopwords.words('english')
    vectorizer = TfidfVectorizer(analyzer='word', stop_words=stopwords_list)
    tfidf_matrix = vectorizer.fit_transform(col)
    similarity = linear_kernel(tfidf_matrix, tfidf_matrix)
    return similarity

In [83]:
pd_info = pds_c[['pd_id', 'brand', 'Price', 'category']].copy()

In [84]:
cat_name = sorted(pd_info['category'].unique())
cat_similarity = pd.DataFrame(compute_similarity(cat_name), index=cat_name, columns=cat_name)
for cat in cat_name:
    cat_df = cat_similarity.nlargest(5, cat)[cat]
    dic = cat_df[(cat_df>0.5) & (cat_df<0.99)].to_dict()
    if len(dic) > 0:
        print(f'{cat}: {dic}')

BB & CC Cream: {'BB & CC Creams': 0.7047887009187664}
BB & CC Creams: {'BB & CC Cream': 0.7047887009187664}
Bath & Body: {'For Body': 0.6324279397451158, 'Bath Soaks & Bubble Bath': 0.599560099075124, 'Bath & Shower': 0.5302029779743187}
Bath & Shower: {'Bath & Body': 0.5302029779743187, 'Bath Soaks & Bubble Bath': 0.5297834333614007}
Bath Soaks & Bubble Bath: {'Bath & Body': 0.599560099075124, 'Bath & Shower': 0.5297834333614007}
Body Lotions & Body Oils: {'For Body': 0.7452428509517302, 'Lotions & Oils': 0.666793141165487}
Body Products: {'For Body': 0.6083589242962121}
Body Sunscreen: {'Face Sunscreen': 0.6485489773755347, 'For Body': 0.6083589242962121}
Brush Cleaners: {'Brush Sets': 0.5433847281822722}
Brush Sets: {'Brush Cleaners': 0.5433847281822722}
Candles: {'Candles & Home Scents': 0.545582838913366}
Candles & Home Scents: {'Candles': 0.545582838913366}
Cologne: {'Cologne Gift Sets': 0.6408163039449903}
Cologne Gift Sets: {'Cologne': 0.6408163039449903, 'Perfume Gift Sets': 0

In [85]:
pd_info['category'] = pd_info['category'].str.replace('BB & CC Creams', 'BB & CC Cream')
pd_info['category'] = pd_info['category'].str.replace('For Body', 'Bath & Body')
pd_info['category'] = pd_info['category'].str.replace('Bath Soaks & Bubble Bath', 'Bath & Body')
pd_info['category'] = pd_info['category'].str.replace('Bath & Shower', 'Bath & Body')
pd_info['category'] = pd_info['category'].str.replace('Brush Sets', 'Brush Cleaners')
pd_info['category'] = pd_info['category'].str.replace('Candles', 'Candles & Home Scents')
pd_info['category'] = pd_info['category'].str.replace('Cologne Gift Sets', 'Perfume Gift Sets')
pd_info['category'] = pd_info['category'].str.replace('Leave-In Conditioner', 'Conditioner')
pd_info['category'] = pd_info['category'].str.replace('Dry Shampoo', 'Shampoo')
pd_info['category'] = pd_info['category'].str.replace('Face Wash', 'Face Wash & Cleansers')
pd_info['category'] = pd_info['category'].str.replace('Hair Removal', 'Hair Removal & Shaving')
pd_info['category'] = pd_info['category'].str.replace('Liquid Lipstick', 'Lipstick')
pd_info['category'] = pd_info['category'].str.replace('Shaving', 'Hair Removal & Shaving')
pd_info['category'] = pd_info['category'].str.replace('Value Sets', 'Value & Gift Sets')

In [114]:
X = np.array([[p] for p in pd_info['Price'].values])

from scipy.spatial import distance_matrix
from sklearn.preprocessing import normalize
price_sim = 1 - normalize(distance_matrix(X, X))

In [122]:
sim_matrix = \
compute_similarity(pds_c['Description']) + \
compute_similarity(pds_c['Name']) + \
compute_similarity(pd_info['brand']) + \
compute_similarity(pd_info['category']) + \
price_sim

In [123]:
xmax, xmin = sim_matrix.max(), sim_matrix.min()
x = (sim_matrix - xmin)/(xmax - xmin)
x

array([[1.        , 0.6368097 , 0.29898584, ..., 0.05715227, 0.04139211,
        0.0436829 ],
       [0.63540858, 1.        , 0.29751333, ..., 0.0658904 , 0.04347683,
        0.04470184],
       [0.29483316, 0.29681042, 1.        , ..., 0.06014343, 0.04297385,
        0.04358231],
       ...,
       [0.05025885, 0.06386113, 0.06008547, ..., 1.        , 0.04731636,
        0.04796331],
       [0.03406376, 0.04115693, 0.04256932, ..., 0.04717825, 1.        ,
        0.46984697],
       [0.03778469, 0.04341083, 0.04424815, ..., 0.04811512, 0.4704283 ,
        1.        ]])

In [125]:
pd.DataFrame(x, columns=pds_c['pd_id'], index=pds_c['pd_id'])

pd_id,P307801,P307804,P460441,P444119,P163604,P377678,P269110,P457233,P457232,P457234,...,P428714,P428708,P428704,P428711,P455881,P428706,P428702,P428709,P452918,P437988
pd_id,,,,,,,,,,,,,,,,,,,,,
P307801,1.000000,0.636810,0.298986,0.317183,0.308102,0.302932,0.300033,0.041968,0.042411,0.042134,...,0.053514,0.057586,0.304218,0.063663,0.062610,0.294815,0.306820,0.057152,0.041392,0.043683
P307804,0.635409,1.000000,0.297513,0.309306,0.348530,0.301878,0.316562,0.042212,0.045661,0.041811,...,0.059118,0.066740,0.305609,0.073673,0.063640,0.301618,0.308427,0.065890,0.043477,0.044702
P460441,0.294833,0.296810,1.000000,0.387838,0.295693,0.285981,0.295870,0.049807,0.057563,0.047192,...,0.056546,0.062625,0.059757,0.066494,0.057233,0.058418,0.062795,0.060143,0.042974,0.043582
P444119,0.310267,0.307239,0.387489,1.000000,0.311172,0.280616,0.291951,0.060043,0.084428,0.115228,...,0.062952,0.078580,0.078502,0.066166,0.068468,0.057451,0.065531,0.080323,0.046455,0.043043
P163604,0.301783,0.346797,0.295467,0.311182,1.000000,0.526008,0.706998,0.050309,0.047437,0.046148,...,0.065609,0.074745,0.069301,0.080203,0.060698,0.069234,0.068535,0.073935,0.045313,0.046881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P428706,0.289801,0.300539,0.058378,0.057599,0.069309,0.139849,0.068512,0.056521,0.063768,0.059426,...,0.445772,0.437823,0.711220,0.487144,0.392012,1.000000,0.645564,0.401744,0.045682,0.046580
P428702,0.301805,0.307349,0.062755,0.065679,0.068610,0.131023,0.056192,0.056008,0.061548,0.059307,...,0.394703,0.394465,0.637118,0.410560,0.385186,0.645564,1.000000,0.378290,0.045208,0.051652
P428709,0.050259,0.063861,0.060085,0.080643,0.074297,0.123714,0.057796,0.061875,0.062730,0.062050,...,0.697662,0.719143,0.385575,0.417319,0.667753,0.401944,0.378490,1.000000,0.047316,0.047963
